In [ ]:
import pandas as pd
import numpy as np
import keras.layers.core as core
import keras.layers.convolutional as conv
import keras.models as models
import keras.utils.np_utils as kutils
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
import pylab
from sklearn.model_selection import KFold

In [ ]:
%matplotlib inline

Данные взяты [отсюда](https://inclass.kaggle.com/c/handwritten-symbols-recognition-cmf).

In [ ]:
train = pd.read_csv('train.csv').values
test = pd.read_csv('test.csv').values
i=0
train[0,:]

Индикаторы классов начинаются с 1. Делаем так, чтобы начинались с нуля.

In [ ]:
y_train = pd.read_csv('train_classes.csv')['class'].values-1
y_train

Загрузим таблицу, сопоставляющую номера классов символам, и посмотрим, какие символы мы должны научится распознавать.

In [ ]:
symbols = pd.read_csv('class_symbols.csv',encoding="cp1251",index_col='class')
i=0
symbols['symbol']

Следующая команда позволяет посмотреть символы выборки.

In [ ]:
print(symbols['symbol'][y_train[i]+1])
pylab.imshow(train[i,:].reshape((48,44)),cmap="Greys_r")
i+=1

Реализуем метрику F1 score для мультиклассовой классификации.

In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def fbeta_score(y_true, y_pred, beta=1):
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score
def fmeasure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=1)

Устанавливаем максимальное количесто эпох, размер батча и размеры входных изображений.

In [ ]:
nb_epoch = 500
batch_size = 30
img_rows, img_cols = 48, 44

Преобразуем элементы выборки в 2d матрицы и отнормируем значения яркости.

In [ ]:
trainX = train[:,:].reshape(train.shape[0], img_rows, img_cols, 1)
trainX = trainX.astype(float)
trainX /= 255.0

То же самое с тестовой частью.

In [ ]:
testX = test[:,:].reshape(test.shape[0], img_rows, img_cols, 1)
testX = testX.astype(float)
testX /= 255.0

Преобразуем целевую переменную в набор дамми-переменных.

In [ ]:
trainY = kutils.to_categorical(y_train)
trainY[0]

Зададим общее число классов.

In [ ]:
nb_classes = trainY.shape[1]

Создаем нейронную сеть данной архитектуры:
<img src="Статья/arch.jpg">

In [ ]:
cnn = models.Sequential()

# Сверточный слой из 20 масок 7 на 7 с функцией активации PReLU.
cnn.add(conv.Convolution2D(20, 7, 7, input_shape=(48, 44, 1), border_mode='valid'))
cnn.add(PReLU())
# Оставляем только максимальное значение в каждом квадрате 2 на 2.
cnn.add(conv.MaxPooling2D(strides=(2,2)))

# Сверточный слой из 50 масок 6 на 6 глубины 20 с функцией активации PReLU.
cnn.add(core.Reshape((1,21,19,20)))
cnn.add(conv.Convolution3D(50, 1, 6, 6, border_mode='valid'))
cnn.add(PReLU())
cnn.add(core.Reshape((16,14,50)))
# Оставляем только максимальное значение в каждом квадрате 2 на 2.
cnn.add(conv.MaxPooling2D(strides=(2,2)))

# Располагаем все сигналы в строчку.
cnn.add(core.Flatten())
# Дропаут слой.
cnn.add(core.Dropout(0.7))

# Полносвязный слой из 330 нейронов с функцией активации PReLU.
cnn.add(core.Dense(330, init = 'glorot_uniform'))
cnn.add(PReLU())

# Нормализирующий и дропаут слои.
cnn.add(BatchNormalization())    
cnn.add(core.Dropout(0.7))

# Полносвязный слой по нейрону на каждый класс.
cnn.add(core.Dense(nb_classes, activation="softmax",init = 'glorot_uniform'))

Эта команда позволяет посмотреть полученную архитектуру сети.

In [ ]:
cnn.summary()

Выбираем loss-функцию, способ обучения сети, а также отслеживаемые метрики качества.

In [ ]:
cnn.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=['accuracy',fmeasure])

Перестаем обучать сеть, если качество на валидационной выборке не улучшилось за последние 50 эпох.

In [ ]:
early_stopping = EarlyStopping(monitor='val_acc', patience=50,verbose=1)

В качестве финальной модели выбираем ту, которая показала лучший результат на валидационной выборке.

In [ ]:
check = ModelCheckpoint('best_model', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')

Разбиваем тренировочную выборку на пять частей и обучаем нейронную сеть построенной архитектуры, используя в качестве валидационной выборки каждую из пяти частей по очереди.

In [ ]:
kf =  KFold(5, shuffle=True,random_state=23)
i=1

for train_index, test_index in kf.split(trainX):
    
    X_train, X_val = trainX[train_index], trainX[test_index]
    y_train, y_val = trainY[train_index], trainY[test_index]
    
    cnn = models.Sequential()

    cnn.add(conv.Convolution2D(20, 7, 7, input_shape=(48, 44, 1), border_mode='valid'))
    cnn.add(PReLU())
    cnn.add(conv.MaxPooling2D(strides=(2,2)))

    cnn.add(core.Reshape((1,21,19,20)))
    cnn.add(conv.Convolution3D(50, 1, 6, 6, border_mode='valid'))
    cnn.add(PReLU())
    cnn.add(core.Reshape((16,14,50)))
    cnn.add(conv.MaxPooling2D(strides=(2,2)))

    cnn.add(core.Flatten())
    cnn.add(core.Dropout(0.7))

    cnn.add(core.Dense(330, init = 'glorot_uniform'))
    cnn.add(PReLU())

    cnn.add(BatchNormalization())    
    cnn.add(core.Dropout(0.7))

    cnn.add(core.Dense(nb_classes, activation="softmax",init = 'glorot_uniform'))
    
    cnn.compile(loss="categorical_crossentropy", optimizer="nadam", metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_acc', patience=50,verbose=0)
    check = ModelCheckpoint('best_model_{}'.format(i), monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')
    
    cnn.fit(X_train, y_train,
        batch_size=batch_size,
        nb_epoch=nb_epoch,
        verbose=0,
        validation_data=(X_val,y_val),
        callbacks=[early_stopping,check],
        shuffle=True)
    
    i = i+1

Загружаем построенные нейронные сети и предсказываем с помощью них вероятности для каждого класса.

In [ ]:
cnn.load_weights("best_model_1")
pred = cnn.predict_proba(testX)
for j in range(2,6):
    cnn.load_weights("best_model_{}".format(j))
    pred = pred+cnn.predict_proba(testX)
pred = pred/5.0
i=0

С помощью этой команды можно посмотреть на тестовые символы и предсказанные алгоритмом значения этих символов.

In [ ]:
w = pred[i,:]
for j in range(5):
    print symbols['symbol'][np.argmax(w)+1],' ',np.max(w)
    w[np.argmax(w)] = 0
pylab.imshow(test[i,:].reshape((48,44)),cmap="Greys_r")
i+=1

Предсказываем класс, не забывая, что мы сместили номера классов на 1 вниз.

In [ ]:
yPred = np.argmax(pred,axis=1)+1

Сохраняем предсказания сети.

In [ ]:
otv = pd.DataFrame({'id':range(0,len(yPred)),'class':yPred})
otv.to_csv('otvet.csv',index=False)

Результаты:

Классификатор|Процент совпадений
---|---
LeNeT-5|61.667
LeNeT-5 + ReLU activation|67.556
Представленная сверточная нейронная сеть|80.556
Человек|84.889